## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanortalik,GL,60.1432,-45.2371,26.49,75,19,42.77,few clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,82.24,70,64,5.17,broken clouds
2,2,Portland,US,45.5234,-122.6762,75.00,23,20,8.05,few clouds
3,3,Talnakh,RU,69.4865,88.3972,-12.08,95,100,20.67,overcast clouds
4,4,Kastrakion,GR,39.7156,21.6181,37.00,79,26,1.57,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,82.24,70,64,5.17,broken clouds
2,2,Portland,US,45.5234,-122.6762,75.00,23,20,8.05,few clouds
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,77.00,65,0,6.91,clear sky
13,13,Kirakira,SB,-10.4544,161.9205,84.00,74,23,6.11,few clouds
16,16,Bambous Virieux,MU,-20.3428,57.7575,78.01,94,75,1.14,light rain
17,17,Carutapera,BR,-1.1950,-46.0200,76.91,92,100,6.76,overcast clouds
18,18,Isangel,VU,-19.5500,169.2667,82.54,76,20,4.25,few clouds
27,27,Hilo,US,19.7297,-155.0900,78.80,53,90,5.75,overcast clouds
28,28,Belmonte,BR,-15.8631,-38.8828,77.00,65,40,6.91,scattered clouds
34,34,Bolong,PH,7.0983,122.2392,75.20,94,40,2.30,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,82.24,broken clouds,-0.6000,73.0833,
2,Portland,US,75.00,few clouds,45.5234,-122.6762,
9,Arraial Do Cabo,BR,77.00,clear sky,-22.9661,-42.0278,
13,Kirakira,SB,84.00,few clouds,-10.4544,161.9205,
16,Bambous Virieux,MU,78.01,light rain,-20.3428,57.7575,
17,Carutapera,BR,76.91,overcast clouds,-1.1950,-46.0200,
18,Isangel,VU,82.54,few clouds,-19.5500,169.2667,
27,Hilo,US,78.80,overcast clouds,19.7297,-155.0900,
28,Belmonte,BR,77.00,scattered clouds,-15.8631,-38.8828,
34,Bolong,PH,75.20,scattered clouds,7.0983,122.2392,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
        
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,82.24,broken clouds,-0.6000,73.0833,Scoop Guest House
2,Portland,US,75.00,few clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
9,Arraial Do Cabo,BR,77.00,clear sky,-22.9661,-42.0278,Pousada Porto Praia
13,Kirakira,SB,84.00,few clouds,-10.4544,161.9205,SanBiz Lodge
16,Bambous Virieux,MU,78.01,light rain,-20.3428,57.7575,Casa Tia Villa
17,Carutapera,BR,76.91,overcast clouds,-1.1950,-46.0200,Lidera
18,Isangel,VU,82.54,few clouds,-19.5500,169.2667,Volcano Sea View Tree House
27,Hilo,US,78.80,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
28,Belmonte,BR,77.00,scattered clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
34,Bolong,PH,75.20,scattered clouds,7.0983,122.2392,Pitas Beach


In [20]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))